In [15]:
import requests as re
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

200

In [17]:
# Extract data from each row
data = []
for i in range(1, 7):
    url = f"https://www.meteoburkina.bf/stations/?page={i}"
    resultat = re.get(url)
    if resultat.status_code==200:
        soupAll = BeautifulSoup(resultat.content, 'lxml')
        soup = soupAll.find("table").find("tbody")
        # Find all table rows
        table_rows = soup.find_all('tr')
        for row in table_rows:
            # Extract text from each table cell (excluding anchor tags)
            cells = [str(td.text.strip()) for td in row.find_all('td') if not td.find('a')]+[row.find("a")["href"]]
            data.append(cells)
        print(f"{url} scrapped and nCells are {len(data)}")
    else:
        print(resultat.status_code)
    time.sleep(random.randint(20,80))

# Create column names based on the first row
columns = [c.text.strip().split("\n")[-1] for c in soupAll.find_all("th",{"class": "orderable"})]#[th.text.strip() for th in table_rows[0].find_all('td')]

df = pd.DataFrame(data, columns=columns)

https://www.meteoburkina.bf/stations/?page=1 scrapped and nCells are 50
https://www.meteoburkina.bf/stations/?page=2 scrapped and nCells are 100
https://www.meteoburkina.bf/stations/?page=3 scrapped and nCells are 150
https://www.meteoburkina.bf/stations/?page=4 scrapped and nCells are 200
https://www.meteoburkina.bf/stations/?page=5 scrapped and nCells are 250
https://www.meteoburkina.bf/stations/?page=6 scrapped and nCells are 252


In [20]:
# Create a pandas DataFrame


In [19]:
df.to_csv("../data/station_meteo.csv")

In [24]:
df[df["Nom"].str.contains("banfora",case=False)]

,Nom,Type,Identificateur WIGOS,Latitude,Longitude,Elevation,Detail url
14,Banfora,Land (fixed),0-854-0-005|primary,"10,636","-4,761",305,https://www.meteoburkina.bf/stations/15


In [26]:
pluie_DF = pd.read_csv("../data/pluie_par_jour.csv")

In [31]:
reg = list(pluie_DF.keys())[1:]

In [34]:
pluie_DF = pluie_DF.rename(columns = {'Année':'Date'})

In [48]:
dfGlob = []
for r in reg:
    df = pd.DataFrame(columns = ["Date", "région","Pluie par année"])
    df["Date"] = pluie_DF["Date"]
    df["région"] = df["Date"].apply(lambda v: r)
    df["Pluie par année"] = pluie_DF[r].apply(lambda v : v)
    dfGlob.append(df)

In [67]:
newDf = pd.concat(dfGlob)

In [68]:
newDf

,Date,région,Pluie par année
0,1996,Hauts-Bassins,84
1,1997,Hauts-Bassins,86
2,1998,Hauts-Bassins,93
3,1999,Hauts-Bassins,99
4,2000,Hauts-Bassins,95
...,...,...,...
22,2018,Burkina Faso,1007
23,2019,Burkina Faso,1025
24,2020,Burkina Faso,955
25,2021,Burkina Faso,993


In [52]:
dfAll = pd.read_csv("../data/ADE_Production Arachide - SUPERFICIE_PRODUCTION.csv")

In [59]:
df_superficie = dfAll[dfAll["indicateur"]=="Superficie"]
df_superficie = df_superficie.rename(columns = {'Value':'Superficie'})

In [77]:
df_Production

,indicateur,région,céréales,Unit,Date,Production
98,Production,Burkina Faso,Arachide,tonne,1996,180 532
99,Production,Burkina Faso,Coton,tonne,1996,150 451
100,Production,Burkina Faso,Maïs,tonne,1996,212 493
101,Production,Burkina Faso,Mil,tonne,1996,733 704
102,Production,Burkina Faso,Niebé,tonne,1996,195 444
...,...,...,...,...,...,...
5284,Production,Sud-Ouest,Maïs,tonne,2022,177 579
5285,Production,Sud-Ouest,Mil,tonne,2022,26 502
5286,Production,Sud-Ouest,Niebé,tonne,2022,30 236
5287,Production,Sud-Ouest,Riz,tonne,2022,20 792


In [60]:
df_Production = dfAll[dfAll["indicateur"]=="Production"]
df_Production = df_Production.rename(columns = {'Value':'Production'})

In [74]:
df_merge = dfAll[["région","céréales","Date"]].drop_duplicates()

In [83]:
def getAllV(line):
    try :
        line["Superficie"] = df_superficie[(df_superficie["région"]==line["région"]) & (df_superficie["céréales"]==line["céréales"]) & (df_superficie["Date"]==line["Date"])]["Superficie"].values.tolist()[0]
    except :
        print("Not found superficie value for ",line)
        line["Superficie"] = 0
    # print(line["Superficie"])
    try:
        line["Production"] = df_Production[(df_Production["région"]==line["région"]) & (df_Production["céréales"]==line["céréales"]) & (df_Production["Date"]==line["Date"])]["Production"].values.tolist()[0]
    except :
        print("Not found Production value for ",line)
        line["Production"] = 0
    try:
        line["Pluie par année"] = newDf[(newDf["région"]==line["région"]) & (newDf["Date"]==line["Date"])]["Pluie par année"].values.tolist()[0]
    except :
        print("Not found Pluie par année value for ",line)
        line["Pluie par année"] = 0
    return line

In [84]:
df_merge = df_merge.apply(lambda l : getAllV(l),axis = 1)

Not found Production value for  région        Sahel
céréales      Coton
Date           2011
Superficie        0
Name: 3024, dtype: object


In [87]:
df_merge.to_csv("../data/all_cereal_merge.csv")

In [97]:
df_merge["céréales"].unique()

array(['Arachide', 'Coton', 'Maïs', 'Mil', 'Niebé', 'Riz', 'Sorgho'],
      dtype=object)

In [94]:
data = {'région': ['Burkina Faso',
  'Boucle du Mouhoun',
  'Cascades',
  'Centre',
  'Centre-Est',
  'Centre-Nord',
  'Centre-Ouest',
  'Centre-Sud',
  'Est',
  'Hauts-Bassins',
  'Nord',
  'Plateau Central',
  'Sahel',
  'Sud-Ouest'],
 'latitude': [12.0753083,
  12.4777805,
  10.30718125,
  12.36732,
  11.734933000000002,
  13.238710000000001,
  11.9067933,
  11.5833646,
  12.255555,
  11.389027500000001,
  13.45911,
  12.40414,
  14.0279165,
  10.38692085],
 'longitude': [-1.6880314,
  -3.5879571310124456,
  -4.435189320350396,
  -1.5430869129979032,
  -0.2921055677472593,
  -1.0349831514084509,
  -2.3044800659164646,
  -1.0556300580745341,
  1.0095763618883151,
  -4.041364986476256,
  -2.2496367403016495,
  -0.8864160224616928,
  -0.7717775666263393,
  -3.283188627811005]}

In [96]:
pd.DataFrame.from_dict(data).to_csv("../data/region_long_lat.csv")